### Finetuning OpenAI Model Exercise

We'll finetune an OpenAI model with custom data to create a custom agent

Inspiration taken from [here](https://www.youtube.com/watch?v=boHXgQ5eQic)

In [150]:
from datasets import load_dataset
import pandas as pd
import numpy as np
pd.options.plotting.backend = "plotly"
import huggingface_hub

from langchain.agents import Tool
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferWindowMemory
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain.agents import initialize_agent, AgentType

List of datasets ready for loading from the [HuggingfaceHub](https://huggingface.co/datasets)

In [106]:
[i for i in huggingface_hub.list_datasets()][:2]

[DatasetInfo(id='acronym_identification', author=None, sha='15ef643450d589d5883e289ffadeb03563e80a9e', created_at=datetime.datetime(2022, 3, 2, 23, 29, 22, tzinfo=datetime.timezone.utc), last_modified=datetime.datetime(2024, 1, 9, 11, 39, 57, tzinfo=datetime.timezone.utc), private=False, gated=False, disabled=False, downloads=1005, likes=17, paperswithcode_id='acronym-identification', tags=['task_categories:token-classification', 'annotations_creators:expert-generated', 'language_creators:found', 'multilinguality:monolingual', 'size_categories:10K<n<100K', 'source_datasets:original', 'language:en', 'license:mit', 'acronym-identification', 'arxiv:2010.14678', 'region:us'], card_data=None, siblings=None),
 DatasetInfo(id='ade_corpus_v2', author=None, sha='4ba01c71687dd7c996597042449448ea312126cf', created_at=datetime.datetime(2022, 3, 2, 23, 29, 22, tzinfo=datetime.timezone.utc), last_modified=datetime.datetime(2024, 1, 9, 11, 42, 58, tzinfo=datetime.timezone.utc), private=False, gated=F

We'll be using the following example [train dataset](https://huggingface.co/datasets/jamescalam/agent-conversations-retrieval-tool). This training messages were generated by GPT-4

**Note:** One can build their own dataset, it is not difficult to implement 

In [10]:
new_data = load_dataset(
    "jamescalam/agent-conversations-retrieval-tool",
    split="train" # which split to return
)

Generating train split: 270 examples [00:00, 41089.30 examples/s]


The training data has 270 messages, each containg a list of roles and content

**Each message represents a single conversation**. Note that all contain the same system message (which we can modify for our particular chatbot we want to train)

In [11]:
new_data

Dataset({
    features: ['messages'],
    num_rows: 270
})

For example, the first message has a set of exchanges between the user and assistant **roles** with **content** examples of how the assistant should behave when prompted with a type of content

In [17]:
print(type(new_data[0]))
new_data[0]

<class 'dict'>


{'messages': [{'role': 'system',
   'content': 'Assistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nAssistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.\n\nOverall, Assistant is a p

In [25]:
print(new_data[0]["messages"][1])

{'role': 'user', 'content': 'TOOLS\n------\nAssistant can ask the user to use tools to look up information that may be helpful in answering the users original question. The tools the human can use are:\n\n> Vector Search Tool: This tool allows you to get research information about LLMs.\n\nRESPONSE FORMAT INSTRUCTIONS\n----------------------------\n\nWhen responding to me, please output a response in one of two formats:\n\n**Option 1:**\nUse this if you want the human to use a tool.\nMarkdown code snippet formatted in the following schema:\n\n```json\n{\n    "action": string, \\ The action to take. Must be one of Vector Search Tool\n    "action_input": string \\ The input to the action\n}\n```\n\n**Option #2:**\nUse this if you want to respond directly to the human. Markdown code snippet formatted in the following schema:\n\n```json\n{\n    "action": "Final Answer",\n    "action_input": string \\ You should put what you want to return to use here\n}\n```\n\nUSER\'S INPUT\n-------------

In [24]:
print(new_data[0]["messages"][1]["content"])

TOOLS
------
Assistant can ask the user to use tools to look up information that may be helpful in answering the users original question. The tools the human can use are:

> Vector Search Tool: This tool allows you to get research information about LLMs.

RESPONSE FORMAT INSTRUCTIONS
----------------------------

When responding to me, please output a response in one of two formats:

**Option 1:**
Use this if you want the human to use a tool.
Markdown code snippet formatted in the following schema:

```json
{
    "action": string, \ The action to take. Must be one of Vector Search Tool
    "action_input": string \ The input to the action
}
```

**Option #2:**
Use this if you want to respond directly to the human. Markdown code snippet formatted in the following schema:

```json
{
    "action": "Final Answer",
    "action_input": string \ You should put what you want to return to use here
}
```

USER'S INPUT
--------------------
Here is the user's input (remember to respond with a markd

Now we convert the training data to json1 format

**Note on JSON1 extension:** One way to store JSON for SQLite: [link](https://www.beekeeperstudio.io/blog/sqlite-json)

In [26]:
new_data.to_json("TRAINING_DATA/jc_conversations_train.json1")

Creating json from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 72.81ba/s]


1103809

### Now that we explored and stored the training data, let's go ahead and train finetune base gpt-3.5

**Note:** If you want to check the validity of the data for training you created, you can do so as stated [here](https://cookbook.openai.com/examples/chat_finetuning_data_prep)

Check out OpenAI's [finetuning training material](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset)

In [30]:
import os
from openai import OpenAI

In [35]:
os.environ["OPENAI_API_KEY"] = ""

client = OpenAI()

We upload our data for training to OpenAI

In [36]:
file_load = client.files.create(
    file=open("TRAINING_DATA/jc_conversations_train.json1", "rb"),
    purpose="fine-tune"
)

In [43]:
file_load

FileObject(id='file-RrC6cpvDUBdq8PA41EumgsG0', bytes=1103809, created_at=1705791884, filename='jc_conversations_train.json1', object='file', purpose='fine-tune', status='processed', status_details=None)

In [50]:
file_load.dict()

{'id': 'file-RrC6cpvDUBdq8PA41EumgsG0',
 'bytes': 1103809,
 'created_at': 1705791884,
 'filename': 'jc_conversations_train.json1',
 'object': 'file',
 'purpose': 'fine-tune',
 'status': 'processed',
 'status_details': None}

We take the file ID and create the finetuning job. We'll **finetune base gpt-3.5-turbo** in this example.

This might take some time to train since it might behind a queue

In [51]:
model_finetune = client.fine_tuning.jobs.create(training_file=file_load.dict()["id"],
                                                model="gpt-3.5-turbo",
                                                suffix="012024_jc_sample")

In [55]:
model_finetune.dict()

{'id': 'ftjob-nUFfwPtcCy6rMJ5ONgi99i4n',
 'created_at': 1705793231,
 'error': None,
 'fine_tuned_model': None,
 'finished_at': None,
 'hyperparameters': {'n_epochs': 'auto',
  'batch_size': 'auto',
  'learning_rate_multiplier': 'auto'},
 'model': 'gpt-3.5-turbo-0613',
 'object': 'fine_tuning.job',
 'organization_id': 'org-HCpyLwy1gf2wzE49sCGQRU79',
 'result_files': [],
 'status': 'validating_files',
 'trained_tokens': None,
 'training_file': 'file-RrC6cpvDUBdq8PA41EumgsG0',
 'validation_file': None}

You can check your **finetuning status** (here)[https://platform.openai.com/finetune] or use the model ID with the API

Check parameter **finished_at**. If None, then it still not finished

In [59]:
client.fine_tuning.jobs.retrieve(model_finetune.dict()["id"]).dict()

{'id': 'ftjob-nUFfwPtcCy6rMJ5ONgi99i4n',
 'created_at': 1705793231,
 'error': None,
 'fine_tuned_model': None,
 'finished_at': None,
 'hyperparameters': {'n_epochs': 3,
  'batch_size': 1,
  'learning_rate_multiplier': 2},
 'model': 'gpt-3.5-turbo-0613',
 'object': 'fine_tuning.job',
 'organization_id': 'org-HCpyLwy1gf2wzE49sCGQRU79',
 'result_files': [],
 'status': 'running',
 'trained_tokens': None,
 'training_file': 'file-RrC6cpvDUBdq8PA41EumgsG0',
 'validation_file': None}

Now it has finished running

In [60]:
client.fine_tuning.jobs.retrieve(model_finetune.dict()["id"]).dict()

{'id': 'ftjob-nUFfwPtcCy6rMJ5ONgi99i4n',
 'created_at': 1705793231,
 'error': None,
 'fine_tuned_model': 'ft:gpt-3.5-turbo-0613:personal:012024-jc-sample:8jFZc3IB',
 'finished_at': 1705795315,
 'hyperparameters': {'n_epochs': 3,
  'batch_size': 1,
  'learning_rate_multiplier': 2},
 'model': 'gpt-3.5-turbo-0613',
 'object': 'fine_tuning.job',
 'organization_id': 'org-HCpyLwy1gf2wzE49sCGQRU79',
 'result_files': ['file-Rshy7mfnCMeqTDgLWyoAqiBf'],
 'status': 'succeeded',
 'trained_tokens': 677358,
 'training_file': 'file-RrC6cpvDUBdq8PA41EumgsG0',
 'validation_file': None}

In [61]:
client.fine_tuning.jobs.retrieve(model_finetune.dict()["id"])

FineTuningJob(id='ftjob-nUFfwPtcCy6rMJ5ONgi99i4n', created_at=1705793231, error=None, fine_tuned_model='ft:gpt-3.5-turbo-0613:personal:012024-jc-sample:8jFZc3IB', finished_at=1705795315, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0613', object='fine_tuning.job', organization_id='org-HCpyLwy1gf2wzE49sCGQRU79', result_files=['file-Rshy7mfnCMeqTDgLWyoAqiBf'], status='succeeded', trained_tokens=677358, training_file='file-RrC6cpvDUBdq8PA41EumgsG0', validation_file=None)

We pull the model name itself

In [63]:
client.fine_tuning.jobs.retrieve(model_finetune.dict()["id"]).dict()["fine_tuned_model"]

'ft:gpt-3.5-turbo-0613:personal:012024-jc-sample:8jFZc3IB'

#### We can take a look at the metrics of the model to see its performance during training, such as its train loss

In [65]:
client.fine_tuning.jobs.retrieve(model_finetune.dict()["id"]).dict()["hyperparameters"]

{'n_epochs': 3, 'batch_size': 1, 'learning_rate_multiplier': 2}

We need to get the result file names from the finetuned object to retrieve their content

In [70]:
result_files = client.fine_tuning.jobs.retrieve(model_finetune.dict()["id"]).dict()["result_files"]
result_files

['file-Rshy7mfnCMeqTDgLWyoAqiBf']

Keep in mind these are metrics from the training procedure, not testing on an external dataset

In [73]:
train_metrics = client.files.retrieve_content(result_files[0])

train_metrics

/var/folders/96/lf090n1d5wz8rqsjs80m1fg80000gn/T/ipykernel_25926/4125317921.py:1: DeprecationWarning: The `.content()` method should be used instead
  train_metrics = client.files.retrieve_content(result_files[0])


'step,train_loss,train_accuracy,valid_loss,valid_mean_token_accuracy\n1,0.72572,0.83133,,\n2,0.64681,0.86667,,\n3,0.51859,0.89809,,\n4,0.33327,0.9403,,\n5,0.22332,0.96815,,\n6,0.31847,0.9403,,\n7,0.31428,0.95,,\n8,0.29801,0.9403,,\n9,0.53667,0.83333,,\n10,0.24963,0.97015,,\n11,0.42805,0.89147,,\n12,0.32082,0.9,,\n13,0.52567,0.91803,,\n14,0.53961,0.91964,,\n15,0.39994,0.87368,,\n16,0.42182,0.87719,,\n17,0.13842,0.97015,,\n18,0.54742,0.88636,,\n19,0.14782,0.95082,,\n20,0.2264,0.9125,,\n21,0.32489,0.93137,,\n22,0.43328,0.91139,,\n23,0.13255,0.96552,,\n24,0.38074,0.94898,,\n25,0.27745,0.91549,,\n26,0.02666,0.98507,,\n27,0.01851,0.98507,,\n28,0.23417,0.9469,,\n29,0.19434,0.9625,,\n30,0.00821,1.0,,\n31,0.31149,0.91781,,\n32,0.08018,0.96809,,\n33,0.14823,0.95062,,\n34,0.15808,0.96226,,\n35,0.12645,0.97101,,\n36,0.16169,0.95699,,\n37,0.73251,0.90816,,\n38,0.15836,0.95294,,\n39,0.00204,1.0,,\n40,0.33968,0.87931,,\n41,0.24433,0.90385,,\n42,0.39227,0.86735,,\n43,0.23507,0.9359,,\n44,0.31601,0.908

In [85]:
train_table = pd.DataFrame([i.split(",") for i in train_metrics.split("\n")][1:], 
                           columns=[i.split(",") for i in train_metrics.split("\n")][0])

train_table

,step,train_loss,train_accuracy,valid_loss,valid_mean_token_accuracy
0,1,0.72572,0.83133,,
1,2,0.64681,0.86667,,
2,3,0.51859,0.89809,,
3,4,0.33327,0.9403,,
4,5,0.22332,0.96815,,
...,...,...,...,...,...
806,807,0.00174,1.0,,
807,808,0.00246,1.0,,
808,809,0.02496,1.0,,
809,810,0.01769,1.0,,


In [105]:
(train_table
 .query("train_loss==train_loss")
 .assign(step = lambda d: pd.to_numeric(d.step),
         train_loss = lambda d: pd.to_numeric(d.train_loss))
 .plot.line(x="step", y="train_loss")
 )

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': 'step=%{x}<br>train_loss=%{y}<extra></extra>',
              'legendgroup': '',
              'line': {'color': '#636efa', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': '',
              'orientation': 'v',
              'showlegend': False,
              'type': 'scatter',
              'x': array([  1,   2,   3, ..., 808, 809, 810]),
              'xaxis': 'x',
              'y': array([0.72572, 0.64681, 0.51859, ..., 0.00246, 0.02496, 0.01769]),
              'yaxis': 'y'}],
    'layout': {'legend': {'tracegroupgap': 0},
               'margin': {'t': 60},
               'template': '...',
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'step'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'train_loss'}}}
})

#### Let's not take a look at loading a using the trained model from OpenAI, using LangChain on RAG

Let's get the finetuned ("ft") model ID

In [110]:
model_name = client.fine_tuning.jobs.retrieve(model_finetune.dict()["id"]).dict()["fine_tuned_model"]
model_name

'ft:gpt-3.5-turbo-0613:personal:012024-jc-sample:8jFZc3IB'

In [111]:
llm = ChatOpenAI(model=model_name, temperature=0.5)

We can see that we loaded it correctly

In [117]:
llm.dict()

{'model_name': 'ft:gpt-3.5-turbo-0613:personal:012024-jc-sample:8jFZc3IB',
 'model': 'ft:gpt-3.5-turbo-0613:personal:012024-jc-sample:8jFZc3IB',
 'stream': False,
 'n': 1,
 'temperature': 0.5,
 '_type': 'openai-chat'}

Setup the memory window. More on how ConversationBufferWindowMemory() works [here](https://github.com/jzamalloa1/langchain_learning/blob/main/Conversation_Memory.ipynb)

**Note of 'memory_key'** The memory_key parameter specifies the key under which the conversation history will be stored and accessed in the memory. In this case, using "messages" as the key means that the conversation history (both the questions asked by the user and the responses given by the bot) will be stored and retrieved using this key. Essentially, it's a label for accessing the stored conversation data. [Source](https://medium.com/@vishalkalia.er/simplifying-langchain-memory-the-power-of-chatbuffermemory-in-chatbots-d44e3911fcb6)

In [119]:
memory_window = ConversationBufferWindowMemory(
    memory_key="chat_history",
    k=5, #The conversation buffer window memory function sets the last number of interactions we want to keep. We are using 2 as an example below
    return_messages=True,
    output_key="output"
    )

We'll load an agent that the arxiv langchain tool. More context can be found [here](https://www.coditation.com/blog/introduction-to-langchain-agents)

In [125]:
arxiv = ArxivQueryRun()

In [146]:
print(arxiv.name)
print(arxiv.description)
print(arxiv.args)

arxiv
A wrapper around Arxiv.org Useful for when you need to answer questions about Physics, Mathematics, Computer Science, Quantitative Biology, Quantitative Finance, Statistics, Electrical Engineering, and Economics from scientific articles on arxiv.org. Input should be a search query.
{'query': {'title': 'Query', 'description': 'search query to look up', 'type': 'string'}}


We initialized a simple tool using the arxiv built-in tool in langchain. More on tools [here](https://python.langchain.com/docs/modules/agents/tools/)

In [147]:
arxiv_tool = Tool(
    name=arxiv.name,
    func=arxiv.run,
    description="useful when you need an answer from Arxiv"
)

We are going to initialize an agent with our model using **initialize_agent. Note that this is now (01/28/24) deprecated.** Refer to more up-to-date langchain agent resources [here](https://python.langchain.com/docs/modules/agents/quick_start). There are langchain classes that will replace this that are more specific to each use case (e.g. create_openai_functions_agent, create_react_agent). I believe [create_react_agent](https://api.python.langchain.com/en/stable/agents/langchain.agents.react.agent.create_react_agent.html#langchain.agents.react.agent.create_react_agent) applies in this case. Will explore later...

In [153]:
my_agent = initialize_agent(
    llm = llm,
    tools=[arxiv_tool],
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    verbose=True,
    max_iterations=3,
    early_stopping_method="generate",
    return_immediate_steps=True,
    memory=memory_window
)

In [154]:
my_agent("Tell me about Llama 2")



> Entering new AgentExecutor chain...
```json
{
    "action": "arxiv",
    "action_input": "Llama 2: A Large Language Model Collection for Code and Instruction-Following Tasks"
}
```
Observation: Published: 2023-08-25
Title: Code Llama: Open Foundation Models for Code
Authors: Baptiste Rozière, Jonas Gehring, Fabian Gloeckle, Sten Sootla, Itai Gat, Xiaoqing Ellen Tan, Yossi Adi, Jingyu Liu, Tal Remez, Jérémy Rapin, Artyom Kozhevnikov, Ivan Evtimov, Joanna Bitton, Manish Bhatt, Cristian Canton Ferrer, Aaron Grattafiori, Wenhan Xiong, Alexandre Défossez, Jade Copet, Faisal Azhar, Hugo Touvron, Louis Martin, Nicolas Usunier, Thomas Scialom, Gabriel Synnaeve
Summary: We release Code Llama, a family of large language models for code based on
Llama 2 providing state-of-the-art performance among open models, infilling
capabilities, support for large input contexts, and zero-shot instruction
following ability for programming tasks. We provide multiple flavors to cover a
wide range of applica

{'input': 'Tell me about Llama 2',
 'chat_history': [HumanMessage(content='Tell me about Llama 2'),
  AIMessage(content='Llama 2 is a collection of large language models that Meta developed and released to the public. It has been fine-tuned for various purposes, such as code generation and instruction-following tasks. The models in Llama 2 show state-of-the-art performance among open models on code benchmarks and Python tasks. There are also specialized variants of Llama 2 for Python and instruction-following tasks, with different parameter sizes. Llama 2 is released under a permissive license that allows for both research and commercial use.')],
 'output': 'Llama 2 is a collection of large language models that Meta developed and released to the public. It has been fine-tuned for various purposes, such as code generation and instruction-following tasks. The models in Llama 2 show state-of-the-art performance among open models on code benchmarks and Python tasks. There are also speciali

Great, the agent was able to run. **Notice** how the output is **similar in format to the training data** that is, the correct JSON format

In [155]:
new_data[0]

{'messages': [{'role': 'system',
   'content': 'Assistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nAssistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.\n\nOverall, Assistant is a p

Let's try with another question

In [156]:
my_agent("How is Llama 2 different than Bard?")



> Entering new AgentExecutor chain...
```json
{
    "action": "arxiv",
    "action_input": "Difference between Llama 2 and Bard"
}
```
Observation: Published: 2023-12-16
Title: A Comparative Analysis of Large Language Models for Code Documentation Generation
Authors: Shubhang Shekhar Dvivedi, Vyshnav Vijay, Sai Leela Rahul Pujari, Shoumik Lodh, Dhruv Kumar
Summary: This paper presents a comprehensive comparative analysis of Large Language
Models (LLMs) for generation of code documentation. Code documentation is an
essential part of the software writing process. The paper evaluates models such
as GPT-3.5, GPT-4, Bard, Llama2, and Starchat on various parameters like
Accuracy, Completeness, Relevance, Understandability, Readability and Time
Taken for different levels of code documentation. Our evaluation employs a
checklist-based system to minimize subjectivity, providing a more objective
assessment. We find that, barring Starchat, all LLMs consistently outperform
the original documenta

{'input': 'How is Llama 2 different than Bard?',
 'chat_history': [HumanMessage(content='Tell me about Llama 2'),
  AIMessage(content='Llama 2 is a collection of large language models that Meta developed and released to the public. It has been fine-tuned for various purposes, such as code generation and instruction-following tasks. The models in Llama 2 show state-of-the-art performance among open models on code benchmarks and Python tasks. There are also specialized variants of Llama 2 for Python and instruction-following tasks, with different parameter sizes. Llama 2 is released under a permissive license that allows for both research and commercial use.'),
  HumanMessage(content='Tell me about Llama 2'),
  AIMessage(content='Llama 2 is a collection of large language models that Meta developed and released to the public. It has been fine-tuned for various purposes, such as code generation and instruction-following tasks. The models in Llama 2 show state-of-the-art performance among o

#### We were able to train our agent. We'll try to learn how to create our data for training in the future (and update the format of agent use from Langchain)